In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random
random.seed(1234)

In [2]:
def get_summary(dat):

    dtypes_dict = dat.dtypes.apply(lambda x: x.name).to_dict()

    numeric = [x for x in dtypes_dict.keys() if dtypes_dict[x] == 'float64' or dtypes_dict[x] == 'int64']
    chars = [x for x in dtypes_dict.keys() if dtypes_dict[x] == 'object']

    type_numeric = ['numeric'] * len(numeric)
    type_chars = ['char'] * len(chars)

    numeric_counts = dat.loc[:, numeric].count().to_dict()
    numeric_populated = list(numeric_counts.values())
    numeric_pct = [str((round(x / len(dat), 2)) * 100) + '%' for x in numeric_populated]
    numeric_zero = [sum(dat.loc[:, x] == 0) for x in numeric]

    numeric_s = pd.DataFrame(list(zip(numeric, type_numeric, numeric_populated, numeric_pct, numeric_zero)),
                             columns=['Name', 'Type', 'Non_NA_Values', 'Pct_Non_NA', 'Num_Zeros'])

    numeric_cols_df = pd.concat([numeric_s,
                                 dat.describe().transpose().reset_index().drop(['count', 'index'], axis=1)],
                                axis=1)

    char_counts = dat.loc[:, chars].count().to_dict()
    char_populated = list(char_counts.values())
    char_pct = [str((round(x / len(dat), 3)) * 100) + '%' for x in char_populated]
    char_zero = [sum(dat.loc[:, x] == 0) for x in chars]

    chars_unique = [len(dat.loc[:, x].unique()) for x in chars]
    chars_val_max = [dat.loc[:, x].value_counts().idxmax() for x in chars]

    chars_df = pd.DataFrame(
        list(zip(chars, type_chars, char_populated, char_pct, char_zero, chars_unique, chars_val_max)),
        columns=['Name', 'Type', 'Non_NA_Values', 'Pct_Non_NA', 'Num_Zeros', 'Unique', 'Most_Common'])

    which = input('Would you like a table of numeric or char columns? (N/C)')
    if which == 'N':
        return numeric_cols_df
    elif which == 'C':
        return chars_df

In [3]:
df = pd.read_csv('Project1_NY property data.csv')

In [4]:
# Repace NAN's with 0's for ZIP only
df['ZIP'] = df['ZIP'].replace(np.nan,0 )

# Groups by Boro and Lot and get the mode
df['c4'] = df.groupby(['B', 'BLOCK'])['ZIP'].transform(lambda x: pd.Series.mode(x)[0])
# Groups by Boro to get MODE
df['c5'] = df.groupby(['B'])['ZIP'].transform(lambda x: pd.Series.mode(x)[0])

# Replaces 0 values with C4 then C5
df['ZIP'] = np.where(df['ZIP'] == 0, 
                      np.where(df['c4'] == 0, df['c5'], df['c4']),
                      df['ZIP'])
# Drops created columns
df = df.drop(['c4', 'c5'], axis = 1)

# Change to categorical
df['ZIP'] = df["ZIP"].astype('int').astype('category')

# Create ZIP 3 as Category
df['ZIP3'] = df['ZIP'].astype(str).str[0:3].astype('category')


In [5]:
# Replaces 0's with NAN for mean gathering. 
df["FULLVAL"] = df["FULLVAL"].replace(0, np.nan)
df["AVLAND"] = df["AVLAND"].replace(0, np.nan)
df["AVTOT"] = df["AVTOT"].replace(0, np.nan)
df["STORIES"] = df["STORIES"].replace(0, np.nan)
df["LTFRONT"] = df["LTFRONT"].replace(0, np.nan)
df["LTDEPTH"] = df["LTDEPTH"].replace(0, np.nan)
df["BLDFRONT"] = df["BLDFRONT"].replace(0, np.nan)
df["BLDDEPTH"] = df["BLDDEPTH"].replace(0, np.nan)

# Fill in 0's for FULLVAL, AVLAND, AVTOT, STORIES, LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH with mean of group
df["FULLVAL"] = df.groupby(['B', 'ZIP'])['FULLVAL'].transform(lambda x: x.fillna(x.mean()))
df["AVLAND"] = df.groupby(['B', 'ZIP'])['AVLAND'].transform(lambda x: x.fillna(x.mean()))
df["AVTOT"] = df.groupby(['B', 'ZIP'])['AVTOT'].transform(lambda x: x.fillna(x.mean()))
df["STORIES"] = df.groupby(['B','ZIP'])['STORIES'].transform(lambda x: x.fillna(x.median()))
df["STORIES"] = df.groupby(['B'])['STORIES'].transform(lambda x: x.fillna(x.median()))
df["LTFRONT"] = df.groupby(['B', 'ZIP'])['LTFRONT'].transform(lambda x: x.fillna(x.median()))
df["LTDEPTH"] = df.groupby(['B', 'ZIP'])['LTDEPTH'].transform(lambda x: x.fillna(x.median()))
df["BLDFRONT"] = df.groupby(['ZIP'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()))
df["BLDDEPTH"] = df.groupby([ 'ZIP'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()))

#Replaces last 0's with BLDGCL global mean

df["BLDFRONT"] = df.groupby(['BLDGCL'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()))
df["BLDDEPTH"] = df.groupby([ 'BLDGCL'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()))

In [6]:
# Create lotarea, bldarea, bldvol
df['lotarea'] = df['LTFRONT'] * df['LTDEPTH']
df['bldarea'] = df['BLDFRONT'] * df['BLDDEPTH']
df['bldvol'] = df['bldarea'] * df['STORIES']

In [7]:
# Create 9 variables

df['FULLVAL_lotarea'] = df['FULLVAL'] / df['lotarea']
df['FULLVAL_bldarea'] = df['FULLVAL'] / df['bldarea']
df['FULLVAL_bldvol'] = df['FULLVAL'] / df['bldvol']
df['AVLAND_Lotarea'] = df['AVLAND'] / df['lotarea']
df['AVLAND_bldarea'] = df['AVLAND'] / df['bldarea']
df['AVLAND_bldvol'] = df['AVLAND'] / df['bldvol']
df['AVTOT_lotarea'] = df['AVTOT'] / df['lotarea']
df['AVTOT_bldarea'] = df['AVTOT'] / df['bldarea']
df['AVTOT_bldvol'] = df['AVTOT'] / df['bldvol']

In [8]:
columns = ['FULLVAL_lotarea','FULLVAL_bldarea', 'FULLVAL_bldvol', 'AVLAND_Lotarea', 'AVLAND_bldarea', \
           'AVLAND_bldvol', 'AVTOT_lotarea', 'AVTOT_bldarea', 'AVTOT_bldvol']

In [9]:
# Loops to create all variable combinations 
for col in columns: 
    df['zip5_' + col] = df[col] / df.groupby('ZIP')[col].transform(lambda x: x.mean())

for col in columns: 
    df['zip3_' + col] = df[col] / df.groupby('ZIP3')[col].transform(lambda x: x.mean())

for col in columns: 
    df['taxclass_' + col] = df[col] / df.groupby('TAXCLASS')[col].transform(lambda x: x.mean())
    
for col in columns: 
    df['borough_' + col] = df[col] / df.groupby('B')[col].transform(lambda x: x.mean())

for col in columns: 
    df['all_' + col] = df[col] / df[col].mean()

In [10]:
df.columns

Index(['RECORD', 'BBLE', 'B', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'ZIP3', 'lotarea', 'bldarea',
       'bldvol', 'FULLVAL_lotarea', 'FULLVAL_bldarea', 'FULLVAL_bldvol',
       'AVLAND_Lotarea', 'AVLAND_bldarea', 'AVLAND_bldvol', 'AVTOT_lotarea',
       'AVTOT_bldarea', 'AVTOT_bldvol', 'zip5_FULLVAL_lotarea',
       'zip5_FULLVAL_bldarea', 'zip5_FULLVAL_bldvol', 'zip5_AVLAND_Lotarea',
       'zip5_AVLAND_bldarea', 'zip5_AVLAND_bldvol', 'zip5_AVTOT_lotarea',
       'zip5_AVTOT_bldarea', 'zip5_AVTOT_bldvol', 'zip3_FULLVAL_lotarea',
       'zip3_FULLVAL_bldarea', 'zip3_FULLVAL_bldvol', 'zip3_AVLAND_Lotarea',
       'zip3_AVLAND_bldarea', 'zip3_AVLAND_bldvol', 'zip3_AVTOT_lotarea',
       'zip3_AVTOT_bldarea', 'zip

In [11]:
get_summary(df.iloc[:, -45:])

,Name,Type,Non_NA_Values,Pct_Non_NA,Num_Zeros,mean,std,min,25%,50%,75%,max
0,zip5_FULLVAL_lotarea,numeric,1070994,100.0%,0,1.0,5.755436,1.715544e-06,0.452184,0.891784,1.203736,2506.256950
1,zip5_FULLVAL_bldarea,numeric,1070994,100.0%,0,1.0,9.999132,2.400407e-06,0.378315,0.842002,1.143872,4534.557129
2,zip5_FULLVAL_bldvol,numeric,1070994,100.0%,0,1.0,11.192967,5.600924e-07,0.346875,0.809131,1.127279,5365.503929
3,zip5_AVLAND_Lotarea,numeric,1070994,100.0%,0,1.0,13.342164,1.911487e-06,0.345289,0.719050,1.061286,9305.941252
4,zip5_AVLAND_bldarea,numeric,1070994,100.0%,0,1.0,21.930206,7.255254e-07,0.181404,0.540334,0.840824,9686.446562
5,zip5_AVLAND_bldvol,numeric,1070994,100.0%,0,1.0,24.193031,2.670986e-07,0.109533,0.464009,0.779872,9247.088104
6,zip5_AVTOT_lotarea,numeric,1070994,100.0%,0,1.0,11.535567,1.029946e-06,0.370859,0.628945,0.936608,7191.342564
7,zip5_AVTOT_bldarea,numeric,1070994,100.0%,0,1.0,19.066755,3.178907e-06,0.303617,0.561409,0.810192,8928.919663
8,zip5_AVTOT_bldvol,numeric,1070994,100.0%,0,1.0,20.568301,1.044773e-06,0.245034,0.518043,0.776108,8604.935283
9,zip3_FULLVAL_lotarea,numeric,1070994,100.0%,0,1.0,8.726385,2.234429e-06,0.429538,0.846049,1.200425,6182.459163


In [12]:
get_summary(df.iloc[:, :-45])

,Name,Type,Non_NA_Values,Pct_Non_NA,Num_Zeros,Unique,Most_Common
0,BBLE,char,1070994,100.0%,0,1070994,5051470022
1,EASEMENT,char,4636,0.4%,0,13,E
2,OWNER,char,1039251,97.0%,0,863349,PARKCHESTER PRESERVAT
3,BLDGCL,char,1070994,100.0%,0,200,R4
4,TAXCLASS,char,1070994,100.0%,0,11,1
5,EXT,char,354305,33.1%,0,4,G
6,STADDR,char,1070318,99.9%,0,839281,501 SURF AVENUE
7,EXMPTCL,char,15579,1.5%,0,15,X1
8,PERIOD,char,1070994,100.0%,0,1,FINAL
9,YEAR,char,1070994,100.0%,0,1,2010/11


In [13]:
df.to_csv('features.csv')